
#### 10.3 Sequence to Sequence (Seq2Seq)

- 구글 기계번역에 사용하는 신경망 모델
- 순차적 정보를 입력받는 신경망(RNN)과 출력하는 신경망 조합.

![개념도](.\img\img4.JPG)

- 입력받기 위한 신경망인 인코더와 출력을 위한 신경망인 디코더로 구성
    - 예: 인코더는 원문, 디코더는 인코더가 번역한 결과물을 입력받음. 그 후 디코더가 출력한 결과물을 번역된 결과물과 비교하면서 학습
    
---------  

** Seq2Seq 모델을 이용해 번역 프로그램을 만들어본다.**
- 네 글다의 영어 단어 입력
- 두 글자의 한글 단어로 번역

![번역모델](.\img\img5.JPG)

- 특수한 심볼이 필요하다
    - 디코더에 입력이 시작됨을 알려주는 심볼, 출력이 끝났음을 알려주는 심볼, 빈 데이터를 채울 때 사용하는 의미없는 빈 심볼 (위 그림에서 'S', 'E', 'P')
    


In [1]:
import tensorflow as tf
import numpy as np


# 글자를 학습시키기위해 원-핫 인코딩 형식으로 변경해야 한다.
# 영어 알파벳과 한글들을 나열 -> 한 글자씩 배열에 삽입
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']

# 배열에 넣은 글자들을 연관배열 형태로 변경
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 학습에 사용할 영단어와 한글단어의 쌍을 가진 데이터 정의
seq_data = [
    ['word','단어'],['wood','나무'],
    ['game','놀이'],['girl','소녀'],
    ['kiss','키스'],['love','사랑']
]



--------
입력, 출력 단어를 하나씩 배열로 만든 후 원-핫 인코딩 형식으로 만들어 주는 함수 생성

1. 인코더 셀의 입력값을 위해 입력 단어를 한 글자씩 떼어 배열로 만든다.
    - input = [num_dic[n] for n in seq[0]]
2. 디코더 셀의 입력값을 위해 출력 단어의 글자들을 배열로 만들고, 시작을 나타내는 심볼 'S'를 맨 앞에 붙인다.
    - output = [num_dic[n] for n in ('S' + seq[1])]
3. 학습을 위해 비교할 디코더 셀의 출력값을 만들고, 출력의 끝을 알려주는 심볼 'E'를 마지막에 붙인다.
    - target = [num_dic[n] for n in (seq[1] + 'E')]
4. 원-핫 인코딩한다.
    - sparse_softmax 함수 사용을 위해 인덱스 숫자 그대로 사용
    

In [2]:

def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    
    return input_batch, output_batch, target_batch

In [3]:

##############
# 신경망 변수
############
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
n_class = n_input = dic_len


In [4]:
#########
# 신경망 모델 구성
############
# 인코더 : [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
# 디코더
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
# 디코더 출력 : [batch size, time steps] 원-핫 인코딩 아니라서 차원이 하나 낮음
targets = tf.placeholder(tf.int64, [None, None])


# 인코더 셀
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell,
                                            output_keep_prob = 0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                           dtype = tf.float32)
# 디코더 셀
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_dell = tf.nn.rnn_cell.DropoutWrapper(dec_cell,
                                            output_keep_prob = 0.5)
    # 초기 상태 값으로 인코더의 최종 상태값 입력(인코더->디코더 전파)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                           initial_state = enc_states, 
                                            dtype = tf.float32)
    


------
고수준 API에서는 가중치와 편향을 알아서 해준다.


In [5]:
# 출력층 생성
model = tf.layers.dense(outputs, n_class, activation = None)

# 손실함수
cost = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = model, labels = targets))

# 최적화
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


In [6]:
#############
# 신경망 학습
##############

sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                      feed_dict = {enc_input : input_batch,
                                  dec_input : output_batch,
                                  targets : target_batch})
    print('Epoch::','%04d' % (epoch + 1),
         'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch:: 0001 cost = 3.787253
Epoch:: 0002 cost = 2.386073
Epoch:: 0003 cost = 1.199667
Epoch:: 0004 cost = 0.612130
Epoch:: 0005 cost = 0.276811
Epoch:: 0006 cost = 0.146814
Epoch:: 0007 cost = 0.073030
Epoch:: 0008 cost = 0.037879
Epoch:: 0009 cost = 0.021718
Epoch:: 0010 cost = 0.013530
Epoch:: 0011 cost = 0.009881
Epoch:: 0012 cost = 0.006638
Epoch:: 0013 cost = 0.003945
Epoch:: 0014 cost = 0.002449
Epoch:: 0015 cost = 0.001665
Epoch:: 0016 cost = 0.001223
Epoch:: 0017 cost = 0.000943
Epoch:: 0018 cost = 0.000750
Epoch:: 0019 cost = 0.000609
Epoch:: 0020 cost = 0.000504
Epoch:: 0021 cost = 0.000423
Epoch:: 0022 cost = 0.000362
Epoch:: 0023 cost = 0.000313
Epoch:: 0024 cost = 0.000275
Epoch:: 0025 cost = 0.000244
Epoch:: 0026 cost = 0.000219
Epoch:: 0027 cost = 0.000199
Epoch:: 0028 cost = 0.000182
Epoch:: 0029 cost = 0.000167
Epoch:: 0030 cost = 0.000155
Epoch:: 0031 cost = 0.000145
Epoch:: 0032 cost = 0.000136
Epoch:: 0033 cost = 0.000128
Epoch:: 0034 cost = 0.000121
Epoch:: 0035 c

In [11]:
##############
# 결과 예측
#############

# 단어를 입력받아 번역 단어를 예측
def translate(word):
    # 예측시에는 한글 데이터를 모름. 따라서 디코더 입출력을 의미없는 P로 채운다.
    seq_data = [word, 'P' * len(word)]
    
    input_batch, output_batch, target_batch = make_batch([seq_data])
    
    
    # 예측 모델을 돌린다.
    
    # 세번째 차원을 argmax로 취해 가장 확률이 높은 글자(인덱스)를 예측값으로 만든다.
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,
                     feed_dict = {enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    # 결과는 인덱스 -> 각 인덱스에 맞는 숫자를 가져와 배열을 만든다.
    decoded = [char_arr[i] for i in result[0]]
    # 출력 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated



예를 들어 
1. 입력으로 'word'를 받으면,  
2. seq_data는 ['word','PPPP']로 구성되고
3. input_batch는 ['w','o','r','d'],  output_batch는 ['P','P','P','P'] 글자들을 원-핫 인코딩한 값일 것.
4. target_batch는 ['P','P','P','P']의 인덱스인 [2,2,2,2]일 것이다.
5. argmax로 예측값을 만든다.
6. 인덱스에 해당하는 글자를 가져와 배열로 만든다.
7. 출력 끝을 의미하는 'E' 이후 글자를 지우고 문자열로 만들어 반환

최종 반환 결과는 ['사','랑','E','E'] 형식으로 나온다.


In [12]:
###########
# 번역 시도
############
print("\n ======= 번역 테스트======")
print('word ->',translate('word'))
print('wodr ->',translate('wodr'))
print('love ->',translate('love'))
print('loev ->',translate('loev'))
print('abcd ->',translate('abcd'))



 ======= 번역 테스트======
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 단어키무



#### 10.4 더보기
Seq2Seq 모델로 만들 수 있는 유용한 응용분야 중 문서 요약이 있다.  
    - Text Summarization
자연어 분석에 사용
    - Word2Vec
